In [45]:
import pandas as pd
import sys
import os
sys.path.append('/home/greg/GitHub/')
import pyEDIutils.search as edi
import datetime
import psycopg2
from sqlalchemy import create_engine
now = datetime.datetime.now().strftime("%Y%m%d")

In [46]:

df = edi.search_pasta(fields=['packageid','title','pubdate','author',
                                   'begindate','enddate','doi'])
df.head()

https://pasta.lternet.edu/package/search/eml?defType=edismax&q=%2A&fq=scope%3Aknb-lter-jrn&fl=packageid%2Ctitle%2Cpubdate%2Cauthor%2Cbegindate%2Cenddate%2Cdoi&sort=packageid%2Casc&rows=500


,packageid,title,pubdate,authors,begindate,enddate,doi
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,2020,"Ji, Wenjie;Hanan, Niall P.",2011-01-01,2011-12-31,doi:10.6073/pasta/313fec8669bc7b4d8debf7393dd2...
1,knb-lter-jrn.101.1,Thresholds and alternative states in neotropic...,2021,"Peinetti, H. Raúl;Bestelmeyer, Brandon;Chirino...",2005-09-01,2018-03-31,doi:10.6073/pasta/e1978467bd09fc4f6e0cd8509b06...
2,knb-lter-jrn.102.1,Carbon exchange responses of rehydrated and in...,2023,"Hoellrich, Mikaela;Pietrasiak, Nicole",2020-01-01,2022-12-31,doi:10.6073/pasta/c6ffd88dc80df1ed1ec32ccdc477...
3,knb-lter-jrn.200021001.5,Data to explore circular manureshed management...,2023,"Spiegal, Sheri;Vendramini, João M.B.;Bittman, ...",2010-01-01,2019-12-31,doi:10.6073/pasta/6025924b89e313f25bddf1912c8a...
4,knb-lter-jrn.200022001.3,Criollo and Crossbred Steer Comparison: Weight...,2022,"McIntosh, Matt M;Cibils, Andres F;Estell, Rick...",2015-12-01,2017-01-31,doi:10.6073/pasta/66ffe320906785bb708c55a61787...


In [47]:
import sqlalchemy as sq
import yaml

# Get the metabase credentials
with open("/home/greg/admin/jrn_metabase_keys.yaml", "r") as stream:
    try:
        mbcred = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Database engine
engine = sq.create_engine("postgresql+psycopg2://{0}:{1}@{2}:{3}/{4}".format(mbcred['user'],
                                                                             mbcred['password'],
                                                                             mbcred['host'],
                                                                             mbcred['port'],
                                                                             mbcred['mbname']))

# connect to the database using the newly created Engine instance
conn = engine.connect()

# run SQL query
mbdf = pd.read_sql_query(sql='SELECT * FROM lter_metabase."DataSet"', con=conn)
mbdf.head()

,DataSetID,Revision,Title,PubDate,Abstract,ShortName,UpdateFrequency,MaintenanceDescription,AbstractType,BoilerplateSetting
0,210011005,105,Annual ground-based photographs taken at 15 ne...,2019-10-09,abstract.210011005.md,None,annually,ongoing,file,jrn-default
1,210086004,105,Leaf litter cover data on 1m x 1m plots from t...,2022-04-11,abstract.210086004.md,None,asNeeded,ongoing - New data are added every 5 years,file,jrn-default
2,210013001,62,Raw neutron counts from a soil water content h...,2019-09-30,abstract.210013001.md,None,asNeeded,ongoing,file,jrn-default
3,210013003,16,Soil volumetric water content calculated from ...,2019-10-29,abstract.210013003.md,None,notPlanned,ongoing,file,jrn-default
4,210011002,109,Seasonal non-destructive vegetation measuremen...,2023-08-23,abstract.210011002.md,None,annually,ongoing,file,jrn-default


In [48]:
df['DataSetID'] = df.packageid.str.split('.', n=2, expand=True).iloc[:,1].astype(int)

In [49]:
df = df.merge(mbdf[['DataSetID','MaintenanceDescription']], on='DataSetID', how='left')

In [50]:
pd.isnull(df.MaintenanceDescription)

0      False
1      False
2      False
3      False
4      False
       ...  
380     True
381    False
382    False
383    False
384    False
Name: MaintenanceDescription, Length: 385, dtype: bool

In [51]:
# Find John's met packages and assign "ongoing"
test = (df['packageid'].str.contains('210437') | df['packageid'].str.contains('210548'))
df.loc[test, 'MaintenanceDescription'] = 'ongoing'
# Find "None" Maintenance descriptions and change to ''
test = pd.isnull(df['MaintenanceDescription'])
df.loc[test, 'MaintenanceDescription'] = ''

In [58]:
import numpy as np
# Create a priority column with empty values
df['priority'] = ''
# Find high priority datasets
test = np.logical_and(df['pubdate'].astype(int) < 2022, df['MaintenanceDescription'].str.contains('ongoing'))
df.loc[test, 'priority'] = 'high'
# Find low priority datasets
test = np.logical_or(df['pubdate'].astype(int) > 2021, df['MaintenanceDescription'].str.contains('complete'))
df.loc[test, 'priority'] = 'low'

In [59]:
df.head()

,packageid,title,pubdate,authors,begindate,enddate,doi,DataSetID,MaintenanceDescription,priority
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,2020,"Ji, Wenjie;Hanan, Niall P.",2011-01-01,2011-12-31,doi:10.6073/pasta/313fec8669bc7b4d8debf7393dd2...,100,ongoing,high
1,knb-lter-jrn.101.1,Thresholds and alternative states in neotropic...,2021,"Peinetti, H. Raúl;Bestelmeyer, Brandon;Chirino...",2005-09-01,2018-03-31,doi:10.6073/pasta/e1978467bd09fc4f6e0cd8509b06...,101,ongoing,high
2,knb-lter-jrn.102.1,Carbon exchange responses of rehydrated and in...,2023,"Hoellrich, Mikaela;Pietrasiak, Nicole",2020-01-01,2022-12-31,doi:10.6073/pasta/c6ffd88dc80df1ed1ec32ccdc477...,102,complete,low
3,knb-lter-jrn.200021001.5,Data to explore circular manureshed management...,2023,"Spiegal, Sheri;Vendramini, João M.B.;Bittman, ...",2010-01-01,2019-12-31,doi:10.6073/pasta/6025924b89e313f25bddf1912c8a...,200021001,complete,low
4,knb-lter-jrn.200022001.3,Criollo and Crossbred Steer Comparison: Weight...,2022,"McIntosh, Matt M;Cibils, Andres F;Estell, Rick...",2015-12-01,2017-01-31,doi:10.6073/pasta/66ffe320906785bb708c55a61787...,200022001,complete,low


In [60]:
df.sort_values(by=['priority','pubdate'], ascending=[True, True])

,packageid,title,pubdate,authors,begindate,enddate,doi,DataSetID,MaintenanceDescription,priority
271,knb-lter-jrn.210520001.1,"Vascular Plant Species of the Jornada Basin, 1...",2018,"Anderson, John;Van Zee, Justin",1982-01-01,2018-09-30,doi:10.6073/pasta/6981502ed9fcf69a6bef71cc133f...,210520001,ongoing,high
28,knb-lter-jrn.210011005.105,Annual ground-based photographs taken at 15 ne...,2019,"Anderson, John",2003-10-14,2018-09-18,doi:10.6073/pasta/69ca7ac7254a2b236d3f44b21fa8...,210011005,ongoing,high
29,knb-lter-jrn.210012001.139,Raw neutron counts from a soil water content h...,2019,"Wierenga, Peter;Virginia, Ross;Duniway, Michae...",1985-11-22,2019-04-30,doi:10.6073/pasta/963587e508b2b01b1bb7f8941f54...,210012001,ongoing,high
30,knb-lter-jrn.210012002.127,Soil volumetric water content calculated from ...,2019,"Wierenga, Peter;Virginia, Ross;Duniway, Michae...",1982-04-30,2019-04-30,doi:10.6073/pasta/2e37c18c14eea1d1e4fccb35ed94...,210012002,ongoing,high
31,knb-lter-jrn.210013001.62,Raw neutron counts from a soil water content h...,2019,"Virginia, Ross;Duniway, Mike;Vivoni, Enrique",1989-07-05,2019-05-03,doi:10.6073/pasta/0f4474b774dab3d097fad39efcf5...,210013001,ongoing,high
...,...,...,...,...,...,...,...,...,...,...
378,knb-lter-jrn.210548101.28,Jornada Basin LTER Cross-scale Interactions St...,2023,"Anderson, John",2017-08-11,2023-09-21,doi:10.6073/pasta/5fa75fe767589be1baaaa57e85de...,210548101,ongoing,low
379,knb-lter-jrn.210548102.27,Jornada Basin LTER Cross-scale Interactions St...,2023,"Anderson, John",2017-08-11,2023-09-21,doi:10.6073/pasta/e1236353b6a15d53aa8864788236...,210548102,ongoing,low
380,knb-lter-jrn.210548103.28,Jornada Basin LTER Cross-scale Interactions St...,2023,"Anderson, John",2017-08-11,2023-09-21,doi:10.6073/pasta/029c896133cb4571259ffe077822...,210548103,ongoing,low
383,knb-lter-jrn.210586001.3,Soil and foliar carbon and nitrogen content an...,2023,"Currier, Courtney;Reichmann, Lara;Sala, Osvaldo E",2011-06-01,2020-09-30,doi:10.6073/pasta/8929d93801db656ae1ad0f3a12a1...,210586001,ongoing,low


In [61]:
df.sort_values(by=['priority','pubdate'], ascending=[True, True]).to_csv('tables_out/jrn_edi_catalog_bypubdate_20231103.csv')